In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_06 import *

## ConvNet

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
x_train,x_valid = normalize_to(x_train,x_valid)

nh,bs = 50,512
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs))

In [ ]:
def get_runner(model, lr=0.6, cbs=None, loss_func = F.cross_entropy):
    opt = optim.SGD(model.parameters(), lr=lr)
    learn = Learner(model, opt, loss_func, data)
    return learn, Runner([AvgStatsCallback([accuracy]), CudaCallback(),
        BatchTransformXCallback(resize_tfm(1,28,28))] + listify(cbs))

In [ ]:
class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None):
        super().__init__()
        self.leak,self.sub,self.maxv = leak,sub,maxv

    def forward(self, x): 
        x = F.leaky_relu(x,self.leak) if self.leak is not None else F.relu(x)
        if self.sub is not None: x.sub_(self.sub)
        if self.maxv is not None: x.clamp_max_(self.maxv)
        return x

In [ ]:
def conv2d(ni, nf, ks=3, stride=2, act=True, **kwargs):
    layers = [nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride)]
    if act: layers.append(GeneralRelu(**kwargs))
    return nn.Sequential(*layers)

In [ ]:
def get_cnn_model(data, nfs, **kwargs):
    nfs = [1] + nfs + [data.c]
    layers = [
        conv2d(nfs[i], nfs[i+1], 5 if i==1 else 3, act=i!=len(nfs)-2, **kwargs)
        for i in range(len(nfs)-1)
    ]
    return nn.Sequential(
        *layers,
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten)
    )

In [ ]:
nfs = [8,16,16,16]

In [ ]:
model = get_cnn_model(data, nfs, leak=0.01, sub=0.5, maxv=6.)
learn,run = get_runner(model, lr=0.7)

In [ ]:
run.fit(3, learn)

train: [1.453769375, tensor(0.4703, device='cuda:0')]
valid: [0.225305712890625, tensor(0.9289, device='cuda:0')]
train: [0.1810441015625, tensor(0.9445, device='cuda:0')]
valid: [0.1402383056640625, tensor(0.9548, device='cuda:0')]
train: [0.116225234375, tensor(0.9638, device='cuda:0')]
valid: [0.094166455078125, tensor(0.9711, device='cuda:0')]


## Export

In [ ]:
!./notebook2script.py 06_cuda_cnn_bn.ipynb

Converted 06_cuda_cnn_bn.ipynb to nb_06.py
